In [27]:
# ETL Pipeline Preparation
### 1. Import libraries and load datasets.
#- Import Python libraries
#- Load `messages.csv` into a dataframe and inspect the first few lines.
#- Load `categories.csv` into a dataframe and inspect the first few lines.

# import libraries
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

# load messages dataset
f =  '../data/disaster_messages.csv'
messages = pd.read_csv(f)


# load categories dataset

f =  '../data/disaster_categories.csv'
categories = pd.read_csv(f)


### 2. Merge datasets.
#- Merge the messages and categories datasets using the common id
#- Assign this combined dataset to `df`, which will be cleaned in the following steps

# merge datasets
df = messages.merge(categories, how='left', on='id')


### 3. Split `categories` into separate category columns.
#- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
#- Use the first row of categories dataframe to create column names for the categories data.
#- Rename columns of `categories` with new column names.

categories = categories.set_index('id').squeeze()

def convert_feature_list(li):
    ''' Converts a string in the format 
        'related-1;request-0;offer-0;
        to a dict where the category names are the keys 
        and the values are the integers.
        i.e. {'related': 1, 'request': 0, 'offer': 0}
        
    '''
    elements = li.split(';')
    return {k.split('-')[0]: int(k.split('-')[-1]) for k in elements}

categories = pd.DataFrame({i: convert_feature_list(v) for i, v in categories.items()}).T


# There are a few entries where the related value is 2. This should be considered a dataerror
# i.e. 
# categories.loc[4145]
# 'related-2;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-0;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-0;floods-0;storm-0;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0'
categories = categories.clip(upper=1)


df = messages.merge(categories, how='left', left_on='id', right_index=True)

### 6. Remove duplicates.

# check number of duplicates
df.duplicated().sum()

# drop duplicates
df.drop_duplicates(inplace=True)

# check number of duplicates
assert not(df.duplicated().any())

### 7. Save the clean dataset into an sqlite database.
engine = create_engine('sqlite:///../data/message_data.db')
df.to_sql('Messages', engine, index=False, if_exists='replace')

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.